#setup

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!pip install transformers

# Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd


from scipy.sparse import hstack
from scipy.sparse import csr_matrix


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'blyat.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "ImageId": dfid["ImageId"],
        "Label": predictions
    })
    submission.head()
    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [ ]:
import warnings
# Filter warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizerFast, BertModel, AdamW
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


In [ ]:
torch.manual_seed(42)
np.random.seed(42)

# specify GPU
device = torch.device("cuda")

In [ ]:
# Reading data
folder_path = '/content/drive/MyDrive/datasets/'
dfTrain = pd.read_csv(folder_path + 'train_cnn.csv')
dfTest = pd.read_csv(folder_path + 'test_cnn.csv')



In [ ]:
dfTest

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader, TensorDataset

def create_dataloader(df, batch_size, is_train=True):
    if is_train:
        # Extracting the labels for training data
        labels = torch.tensor(df['label'].values, dtype=torch.long)
    else:
        # For testing data, labels are not available
        labels = torch.zeros(len(df), dtype=torch.long)  # Placeholder; won't be used

    # Extracting the image pixels
    images = torch.tensor(df.drop(columns=['label'] if is_train else []).values, dtype=torch.float32)
    images = images.view(-1, 1, 28, 28)  # Reshaping to match the expected input shape

    # Creating a TensorDataset and DataLoader
    dataset = TensorDataset(images, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=is_train)

    return dataloader



#REALLY SUPER POOPER COOL STUFF

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN_Arch(nn.Module):
    def __init__(self, num_classes=10, activation='relu', dropout=0.5):
        super(CNN_Arch, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)

        # Max pooling layer
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 5 * 5, 256)  # Updated input size
        self.fc2 = nn.Linear(256, num_classes)  # Output size is the number of classes (digits 0-9)

        # Choose activation function
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'gelu':
            self.activation = nn.GELU()

        # Dropout layer with specified rate
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Convolutional layers with chosen activation function and max pooling
        x = self.max_pool(self.activation(self.conv1(x)))  # Shape: (32, 13, 13)
        x = self.max_pool(self.activation(self.conv2(x)))  # Shape: (64, 5, 5)

        # Flatten the tensor
        x = torch.flatten(x, 1)  # Shape: (64 * 5 * 5)

        # Fully connected layers with chosen activation function and dropout
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [ ]:
class CNN_Arch(nn.Module):
    def __init__(self, num_classes=10, activation='relu', dropout=0.5):
        super(CNN_Arch, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        # Max pooling layer
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Batch normalization layers
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

        # Choose activation function
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'gelu':
            self.activation = nn.GELU()


        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.max_pool(self.activation(self.bn1(self.conv1(x))))
        x = self.max_pool(self.activation(self.bn2(self.conv2(x))))
        x = self.activation(self.bn3(self.conv3(x)))
        x = self.max_pool(self.activation(self.bn4(self.conv4(x))))
        x = self.activation(self.bn5(self.conv5(x)))

        # Flatten the tensor
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
class LSTM_Arch(nn.Module):
    def __init__(self, input_size,
                 hidden_sizes,  # List of hidden sizes
                 dropouts,      # List of dropouts
                 attention_size):
        super(LSTM_Arch, self).__init__()
        num_layers = len(hidden_sizes)  # Determine the number of layers from the hidden_sizes list
        self.lstms = nn.ModuleList([nn.LSTM(input_size if i == 0 else hidden_sizes[i - 1],
                                            hidden_sizes[i],
                                            num_layers=1,
                                            batch_first=True,
                                            dropout=0)  # Set dropout to 0 since num_layers is 1
                                    for i in range(num_layers)])
        self.attentions = nn.ModuleList([nn.Linear(hidden_sizes[i], attention_size) for i in range(num_layers)])
        self.attention_combines = nn.ModuleList([nn.Linear(attention_size, 1) for i in range(num_layers)])
        self.fc = nn.Linear(hidden_sizes[-1], 1)



    def apply_attention(self, out, attention_layer, attention_combine_layer):
        attention_weights = F.softmax(attention_combine_layer(F.tanh(attention_layer(out))), dim=1)
        out = torch.bmm(attention_weights.transpose(1, 2), out)
        return out.squeeze(1)

    def forward(self, x):
        for i in range(len(self.lstms)):3
            out, _ = self.lstms[i](x if i == 0 else out.unsqueeze(1))
            out = self.apply_attention(out, self.attentions[i], self.attention_combines[i])
        out = self.fc(out)
        return out

In [ ]:
def predict(model, test_dataloader,device):# Get the device from the model
    model.eval() # Set the model to evaluation mode
    predictions = []

    with torch.no_grad(): # No need to compute gradients during prediction
        for images, _ in test_dataloader: # Unpack images and ignore labels
            images = images.to(device) # Move images to the device
            outputs = model(images) # Get model predictions
            _, predicted_labels = torch.max(outputs, 1) # Get the index of the max value
            predictions.extend(predicted_labels.cpu().numpy()) # Store predictions

    return predictions
import random

In [ ]:
from sklearn.metrics import classification_report

def train(model, optimizer, train_dataloader, device, loss_function):
    model.train()
    total_loss = 0
    correct_predictions = 0
    true_labels_train = []  # Store true labels
    predicted_labels_train = []  # Store predicted labels

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        model.zero_grad()
        predictions = model(images)
        loss = loss_function(predictions, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted_labels = torch.max(predictions, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        true_labels_train.extend(labels.cpu().numpy())  # Add true labels to list
        predicted_labels_train.extend(predicted_labels.cpu().numpy())  # Add predicted labels to list

    avg_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / len(train_dataloader.dataset)

    return avg_loss, accuracy, true_labels_train, predicted_labels_train

def evaluate(model, val_dataloader, device, loss_function):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    true_labels_eval = []  # Store true labels
    predicted_labels_eval = []  # Store predicted labels

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            predictions = model(images)
            loss = loss_function(predictions, labels)
            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_labels = torch.max(predictions, 1)
            correct_predictions += (predicted_labels == labels).sum().item()
            true_labels_eval.extend(labels.cpu().numpy())  # Add true labels to list
            predicted_labels_eval.extend(predicted_labels.cpu().numpy())  # Add predicted labels to list

    avg_loss = total_loss / len(val_dataloader)
    accuracy = correct_predictions / len(val_dataloader.dataset)

    return avg_loss, accuracy, true_labels_eval, predicted_labels_eval

best_state_dict = None
best_params = None
best_valid_metric = 0
def objective(trial, dfTrain, device, layers=3):
    global best_state_dict, best_params,best_valid_metric

    model_params = {
    "lr": [3e-5, 1e-2],
    "weight_decay": [1e-6, 1e-2],
    "batch_size": [40, 50],
    "epochs": [5,10],
    "beta1": [0.8, 0.99],
    "beta2": [0.9, 0.9999],
    "factor": [0.05, 0.5],
    "scheduler_patience": [1, 2],
    "threshold": [1e-5, 1e-3],
    "cooldown": [0, 1],
    "min_lr": [0, 1e-3],
    "dropout": [0.1, 0.5],  # Only one definition of "dropout" is needed
    "activation": ['leaky_relu', 'sigmoid', 'tanh', 'gelu'],
    'test_size':[0.35,0.45],
    'train_size':[0.5,0.75],
    'random_state':[42,1024],
    }
    params = {}
    for param, bounds in model_params.items():
        if isinstance(bounds[0], int):
            params[param] = trial.suggest_int(param, min(bounds), max(bounds))
        elif isinstance(bounds[0], bool) or isinstance(bounds[0], str): # Handle strings
            params[param] = trial.suggest_categorical(param, bounds)
        elif isinstance(bounds[0], float):
            params[param] = trial.suggest_float(param, min(bounds), max(bounds), log=True)

    for param, value in params.items():
        print(f'{param} = {value}')

    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    train_data, val_data_temp = train_test_split(dfTrain, test_size=1- params["train_size"], random_state=params["random_state"])
    y_train,y_val_temp=train_test_split(dfTrain['label'], test_size=1- params["train_size"], random_state=params["random_state"])

    test_data, val_data = train_test_split(val_data_temp, test_size=1- params["test_size"], random_state=params["random_state"])
    y_test, val_y=train_test_split(y_val_temp, test_size=1-params["test_size"], random_state=params["random_state"])




    train_dataloader = create_dataloader(train_data, batch_size=params["batch_size"])
    val_dataloader = create_dataloader(val_data, batch_size=params["batch_size"])
    test_dataloader = create_dataloader(test_data, batch_size=params["batch_size"])






    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    val_labels,train_labels = train_test_split(dfTrain['label'], test_size=params["test_size"],random_state=params["random_state"]+random.randint(0,300),stratify=dfTrain['label'])
    #all_labels = np.concatenate([train_labels, val_labels])
    all_labels=train_labels
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
    weights = torch.tensor(class_weights, dtype=torch.float)
    weights = weights.to(device)


    #loss_function = nn.NLLLoss(weight=weights)
    loss_function = nn.CrossEntropyLoss(weight=weights)

    best_local_metric = 0
    valid_losses = []
    no_improvement_counter = 0
    for epoch in range(1000000):
        train_loss, train_accuracy, true_labels_train, predicted_labels_train = train(model, optimizer, train_dataloader, device, loss_function)
        valid_loss, val_accuracy, true_labels_eval, predicted_labels_eval = evaluate(model, val_dataloader, device, loss_function)
        if test_dataloader is not None:
            test_preds = predict(model, test_dataloader,device)
            print("Testing Classification Report:")
            test_accuracy=accuracy_score(y_test, test_preds)
            print('-' * 107); print(f'| Epoch {epoch + 1} | Loss Train/Valid: {train_loss:.6f} / {valid_loss:.6f} |', end=''); print(f' Accuracy Train/Valid: {100 * train_accuracy:.3f}% / {100 * val_accuracy:.4f}% / {100 * test_accuracy:.4f}% |'); print('-' * 107);
            print()
        else:
            print('-' * 100); print(f'| Epoch {epoch + 1} | Loss Train/Valid: {train_loss:.6f} / {valid_loss:.6f} |', end=''); print(f' Accuracy Train/Valid: {100 * train_accuracy:.3f}% / {100 * val_accuracy:.4f}% |'); print('-' * 100);
        #metric = valid_loss
        metric=train_accuracy+val_accuracy+test_accuracy
        if metric > best_local_metric:
            best_local_metric = metric
            no_improvement_counter = 0
            if metric > best_valid_metric:
                best_valid_metric = metric
                best_state_dict = copy.deepcopy(model.state_dict())
                best_params=params
                print();print('~'*26);print(f'|Best Value Found {best_local_metric:.5f}|');print('~'*26); print(f'Params = {best_params}'); print('~' * 54)
                print("Training Classification Report:"); print(classification_report(true_labels_train, predicted_labels_train))
                print("Validation Classification Report:"); print(classification_report(true_labels_eval, predicted_labels_eval))
                print('~' * 54); print()
                if test_dataloader is not None:
                    test_preds = predict(model, test_dataloader,device)
                    print("Testing Classification Report:")
                    print(classification_report(y_test, test_preds))
                    print('~' * 54)
                    print()
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= 2:
                print(); print('<'*50); print('OH NO WE BE OVERFITTING'); print('<'*50); print()
                raise optuna.TrialPruned()
    return metric


def tune_model_with_optuna(dfTrain, device, layers=3):
    global best_state_dict, best_params

    study = optuna.create_study(direction="maximize")  # Maximize the accuracy
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    # Load the best model using the best state dictionary
    best_model = CNN_Arch(
        dropout=best_params["dropout"] ,
        activation=best_params["activation"],# Specify other parameters as needed
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)

    return best_model



In [ ]:
import copy
from sklearn.metrics import accuracy_score

In [ ]:

best_state_dict = None
best_params = None
best_valid_metric = 0
def objective(trial, dfTrain, device, layers=3):
    global best_state_dict, best_params,best_valid_metric

    model_params = {
    "lr": [3e-5, 1e-2],
    "weight_decay": [1e-6, 1e-2],
    "batch_size": [40, 50],
    "epochs": [5,10],
    "beta1": [0.8, 0.99],
    "beta2": [0.9, 0.9999],
    "factor": [0.05, 0.5],
    "scheduler_patience": [1, 2],
    "threshold": [1e-5, 1e-3],
    "cooldown": [0, 1],
    "min_lr": [0, 1e-3],
    "dropout": [0.1, 0.5],  # Only one definition of "dropout" is needed
    "activation": ['leaky_relu', 'sigmoid', 'tanh', 'gelu'],
    'test_size':[0.35,0.45],
    'train_size':[0.5,0.75],
    'random_state':[42,1024],
    }
    params = {}
    for param, bounds in model_params.items():
        if isinstance(bounds[0], int):
            params[param] = trial.suggest_int(param, min(bounds), max(bounds))
        elif isinstance(bounds[0], bool) or isinstance(bounds[0], str): # Handle strings
            params[param] = trial.suggest_categorical(param, bounds)
        elif isinstance(bounds[0], float):
            params[param] = trial.suggest_float(param, min(bounds), max(bounds), log=True)

    for param, value in params.items():
        print(f'{param} = {value}')

    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    train_data, val_data_temp = train_test_split(dfTrain, test_size=1- params["train_size"], random_state=params["random_state"])
    y_train,y_val_temp=train_test_split(dfTrain['label'], test_size=1- params["train_size"], random_state=params["random_state"])

    test_data, val_data = train_test_split(val_data_temp, test_size=1- params["test_size"], random_state=params["random_state"])
    y_test, val_y=train_test_split(y_val_temp, test_size=1-params["test_size"], random_state=params["random_state"])




    train_dataloader = create_dataloader(train_data, batch_size=params["batch_size"])
    val_dataloader = create_dataloader(val_data, batch_size=params["batch_size"])
    test_dataloader = create_dataloader(test_data, batch_size=params["batch_size"])






    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    val_labels,train_labels = train_test_split(dfTrain['label'], test_size=params["test_size"],random_state=params["random_state"]+random.randint(0,300),stratify=dfTrain['label'])
    #all_labels = np.concatenate([train_labels, val_labels])
    all_labels=train_labels
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
    weights = torch.tensor(class_weights, dtype=torch.float)
    weights = weights.to(device)


    #loss_function = nn.NLLLoss(weight=weights)
    loss_function = nn.CrossEntropyLoss(weight=weights)

    best_local_metric = 0
    valid_losses = []
    no_improvement_counter = 0
    for epoch in range(1000000):
        train_loss, train_accuracy, true_labels_train, predicted_labels_train = train(model, optimizer, train_dataloader, device, loss_function)
        valid_loss, val_accuracy, true_labels_eval, predicted_labels_eval = evaluate(model, val_dataloader, device, loss_function)
        if test_dataloader is not None:
            test_preds = predict(model, test_dataloader,device)
            print("Testing Classification Report:")
            test_accuracy=accuracy_score(y_test, test_preds)
            print('-' * 107); print(f'| Epoch {epoch + 1} | Loss Train/Valid: {train_loss:.6f} / {valid_loss:.6f} |', end=''); print(f' Accuracy Train/Valid: {100 * train_accuracy:.3f}% / {100 * val_accuracy:.4f}% / {100 * test_accuracy:.4f}% |'); print('-' * 107);
            print()
        else:
            print('-' * 100); print(f'| Epoch {epoch + 1} | Loss Train/Valid: {train_loss:.6f} / {valid_loss:.6f} |', end=''); print(f' Accuracy Train/Valid: {100 * train_accuracy:.3f}% / {100 * val_accuracy:.4f}% |'); print('-' * 100);
        #metric = valid_loss
        metric=train_accuracy+val_accuracy+test_accuracy
        if metric > best_local_metric:
            best_local_metric = metric
            no_improvement_counter = 0
            if metric > best_valid_metric:
                best_valid_metric = metric
                best_state_dict = copy.deepcopy(model.state_dict())
                best_params=params
                print();print('~'*26);print(f'|Best Value Found {best_local_metric:.5f}|');print('~'*26); print(f'Params = {best_params}'); print('~' * 54)
                print("Training Classification Report:"); print(classification_report(true_labels_train, predicted_labels_train))
                print("Validation Classification Report:"); print(classification_report(true_labels_eval, predicted_labels_eval))
                print('~' * 54); print()
                if test_dataloader is not None:
                    test_preds = predict(model, test_dataloader,device)
                    print("Testing Classification Report:")
                    print(classification_report(y_test, test_preds))
                    print('~' * 54)
                    print()
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= 2:
                print(); print('<'*50); print('OH NO WE BE OVERFITTING'); print('<'*50); print()
                raise optuna.TrialPruned()
    return metric


def tune_model_with_optuna(dfTrain, device, layers=3):
    global best_state_dict, best_params

    study = optuna.create_study(direction="maximize")  # Maximize the accuracy
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    # Load the best model using the best state dictionary
    best_model = CNN_Arch(
        dropout=best_params["dropout"] ,
        activation=best_params["activation"],# Specify other parameters as needed
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)

    return best_model



In [ ]:
tuned_model=tune_model_with_optuna(dfTrain, device)

[I 2023-08-16 00:37:33,341] A new study created in memory with name: no-name-7cf95a9d-773e-436a-be9e-e52c30658fd4


lr = 0.0004136875361759556
weight_decay = 0.003996303491483556
batch_size = 46
epochs = 6
beta1 = 0.9487914163580693
beta2 = 0.9457495961583618
factor = 0.1340385681811893
scheduler_patience = 2
threshold = 5.594129250116556e-05
cooldown = 1
min_lr = 0
dropout = 0.2618875587975504
activation = tanh
test_size = 0.44978099873399513
train_size = 0.5968457154065806
random_state = 230
Testing Classification Report:
-----------------------------------------------------------------------------------------------------------
| Epoch 1 | Loss Train/Valid: 0.211408 / 0.077088 | Accuracy Train/Valid: 93.541% / 97.7997% / 10.1103% |
-----------------------------------------------------------------------------------------------------------


~~~~~~~~~~~~~~~~~~~~~~~~~~
|Best Value Found 2.01451|
~~~~~~~~~~~~~~~~~~~~~~~~~~
Params = {'lr': 0.0004136875361759556, 'weight_decay': 0.003996303491483556, 'batch_size': 46, 'epochs': 6, 'beta1': 0.9487914163580693, 'beta2': 0.9457495961583618, 'factor': 0.134

[I 2023-08-16 00:39:04,109] Trial 0 pruned. 


Testing Classification Report:
-----------------------------------------------------------------------------------------------------------
| Epoch 15 | Loss Train/Valid: 0.010103 / 0.063587 | Accuracy Train/Valid: 99.677% / 98.8301% / 9.9527% |
-----------------------------------------------------------------------------------------------------------


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
OH NO WE BE OVERFITTING
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

lr = 0.0067811437740502595
weight_decay = 5.812072677383952e-05
batch_size = 48
epochs = 9
beta1 = 0.8659390254855196
beta2 = 0.9053996439021338
factor = 0.21542906076757362
scheduler_patience = 2
threshold = 4.917690321746687e-05
cooldown = 1
min_lr = 0
dropout = 0.20391941254048238
activation = gelu
test_size = 0.43134099081493166
train_size = 0.5269146977989723
random_state = 601
Testing Classification Report:
-----------------------------------------------------------------------------------------------------

[W 2023-08-16 00:39:30,925] Trial 1 failed with parameters: {'lr': 0.0067811437740502595, 'weight_decay': 5.812072677383952e-05, 'batch_size': 48, 'epochs': 9, 'beta1': 0.8659390254855196, 'beta2': 0.9053996439021338, 'factor': 0.21542906076757362, 'scheduler_patience': 2, 'threshold': 4.917690321746687e-05, 'cooldown': 1, 'min_lr': 0, 'dropout': 0.20391941254048238, 'activation': 'gelu', 'test_size': 0.43134099081493166, 'train_size': 0.5269146977989723, 'random_state': 601} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-20-d4f471ba35b9>", line 123, in <lambda>
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
  File "<ipython-input-20-d4f471ba35b9>", line 82, in objective
    train_loss, train_accuracy, true_labels_train, predicted_labels_train 

KeyboardInterrupt: ignored

###fdfsf

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn.functional as F


from math import sqrt

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn.functional as F


from math import sqrt

import copy
best_state_dict = None
best_params = None
best_valid_metric = 0
def objective(trial, dfTrain, device, layers=3):
    global best_state_dict, best_params,best_valid_metric

    model_params = {
    "lr": [3e-5, 1e-2],
    "weight_decay": [1e-6, 1e-2],
    "batch_size": [40, 50],
    "epochs": [5,10],
    "beta1": [0.8, 0.99],
    "beta2": [0.9, 0.9999],
    "factor": [0.05, 0.5],
    "scheduler_patience": [1, 2],
    "threshold": [1e-5, 1e-3],
    "cooldown": [0, 1],
    "min_lr": [0, 1e-3],
    "dropout": [0.1, 0.5],  # Only one definition of "dropout" is needed
    "activation": ['leaky_relu', 'sigmoid', 'tanh', 'gelu'],
    'test_size':[0.15,0.45],
    'random_state':[42,1024],
}
    params = {}
    for param, bounds in model_params.items():
        if isinstance(bounds[0], int):
            params[param] = trial.suggest_int(param, min(bounds), max(bounds))
        elif isinstance(bounds[0], bool) or isinstance(bounds[0], str): # Handle strings
            params[param] = trial.suggest_categorical(param, bounds)
        elif isinstance(bounds[0], float):
            params[param] = trial.suggest_float(param, min(bounds), max(bounds), log=True)

    for param, value in params.items():
        print(f'{param} = {value}')

    train_data, val_data = train_test_split(dfTrain, test_size=params["test_size"], random_state=params["random_state"])
    y_train,y_val=train_test_split(dfTrain['label'], test_size=params["test_size"], random_state=params["random_state"])
    #train_data, val_data = train_test_split(dfTrain, test_size=0.3, random_state=52)
    #y_train,y_val=train_test_split(dfTrain['label'], test_size=0.3, random_state=52)

    # Create data loaders for training and validation data
    train_dataloader = create_dataloader(train_data, batch_size=params["batch_size"])
    val_dataloader = create_dataloader(val_data, batch_size=params["batch_size"])  # Using the same function for validation data

    # Instantiate the model using the new CNN_Arch class
    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])

    model = model.to(device)


    optimizer = AdamW(model.parameters(),
                      lr=params["lr"],
                      betas=(params["beta1"], params["beta2"]),
                      #eps=params["eps_optimizer"],
                      weight_decay=params["weight_decay"])

    scheduler = ReduceLROnPlateau(optimizer,
                                  mode='min',
                                  factor=params["factor"],
                                  patience=params["scheduler_patience"],
                                  verbose=False,
                                  threshold=params["threshold"],
                                  threshold_mode='rel',
                                  cooldown=params["cooldown"],
                                  min_lr=params["min_lr"],
                                  #eps=params["scheduler_eps"]
                                  )

    loss_function = nn.CrossEntropyLoss()
    best_local_metric=0

    separator = '=' * 180
    valid_losses=[]



    for epoch in range(params["epochs"]):
        print('-'*14)
        print(f'|Epoch {epoch + 1} / {params["epochs"]}|')
        print('-'*31)
        train_loss, train_accuracy, true_labels_train, predicted_labels_train = train(model, optimizer, train_dataloader, device, loss_function)
        valid_loss, val_accuracy, true_labels_eval, predicted_labels_eval = evaluate(model, val_dataloader, device, loss_function)

        print(f'|Loss Train/Valid:   {train_loss:.4f}  /  {valid_loss:.4f}  |')
        print(f'|Accuracy Train/Valid:   {100*train_accuracy:.4f}%  /  {100*val_accuracy:.4f}%  |')
        print('-'*47)

        if val_accuracy >    best_local_metric:
            best_local_metric=val_accuracy
            if best_local_metric > best_valid_metric:
                best_valid_metric = val_accuracy
                best_state_dict = copy.deepcopy(model.state_dict())
                best_params = params
                print()
                print('^'*100)
                print('MODEL HAVE BEEN SAVED BEAWARE')
                print('^'*100)
                print()
        valid_losses.append(valid_loss)

    print("Training Classification Report:")
    print(classification_report(true_labels_train, predicted_labels_train))
    print("Validation Classification Report:")
    print(classification_report(true_labels_eval, predicted_labels_eval))

    print('*' * 180)
    print('                                                                                    TRIAL=GUD')
    print('*' * 180)
    print()


    torch.save(model.state_dict(), f'saved_weights_trial_{trial.number}.pt')
    avg_valid_loss = sum(valid_losses) / len(valid_losses)

    print(('-----------------------'))



    return val_accuracy



def tune_model_with_optuna(dfTrain, device, layers=3):
    global best_state_dict, best_params

    study = optuna.create_study(direction="maximize")  # Maximize the accuracy
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    # Load the best model using the best state dictionary
    best_model = CNN_Arch(
        dropout=best_params["dropout"] ,
        activation=best_params["activation"],# Specify other parameters as needed
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)

    return best_model


#here

In [ ]:
best_state_dict = None
best_params = None
best_valid_metric = 0

def tune_model_with_optuna(dfTrain, device, layers=3):
    global best_state_dict, best_params

    study = optuna.create_study(direction="minimize")  # Maximize the accuracy
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    # Load the best model using the best state dictionary
    best_model = CNN_Arch(
        dropout=best_params["dropout"] ,
        activation=best_params["activation"],# Specify other parameters as needed
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)

    return best_model

tuned_model=tune_model_with_optuna(dfTrain, device)

[I 2023-08-14 01:23:49,466] A new study created in memory with name: no-name-ec35dfdc-fcf3-4c4b-af84-8b1994e4cb48


lr = 0.007435501294031588
weight_decay = 1.2268231483339016e-05
batch_size = 44
epochs = 7
beta1 = 0.8569527533160558
beta2 = 0.9121194410267074
factor = 0.3389174191734056
scheduler_patience = 2
threshold = 0.00010591442586341901
cooldown = 1
min_lr = 0
dropout = 0.34710101063456267
activation = gelu
test_size = 0.32379792894039017
random_state = 898
--------------
|Epoch 1 / 7|
-------------------------------
|Loss Train/Valid:   0.6597  /  0.2628  |
|Accuracy Train/Valid:   85.3134%  /  94.3603%  |
-----------------------------------------------

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
MODEL HAVE BEEN SAVED BEAWARE
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

--------------
|Epoch 2 / 7|
-------------------------------
|Loss Train/Valid:   0.2467  /  0.4395  |
|Accuracy Train/Valid:   95.1127%  /  93.8676%  |
-----------------------------------------------
----------

[I 2023-08-14 01:24:32,420] Trial 0 finished with value: 0.9847794117647058 and parameters: {'lr': 0.007435501294031588, 'weight_decay': 1.2268231483339016e-05, 'batch_size': 44, 'epochs': 7, 'beta1': 0.8569527533160558, 'beta2': 0.9121194410267074, 'factor': 0.3389174191734056, 'scheduler_patience': 2, 'threshold': 0.00010591442586341901, 'cooldown': 1, 'min_lr': 0, 'dropout': 0.34710101063456267, 'activation': 'gelu', 'test_size': 0.32379792894039017, 'random_state': 898}. Best is trial 0 with value: 0.9847794117647058.


|Loss Train/Valid:   0.2355  /  0.1204  |
|Accuracy Train/Valid:   97.8380%  /  98.4779%  |
-----------------------------------------------

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
MODEL HAVE BEEN SAVED BEAWARE
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Training Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2801
           1       0.99      0.98      0.99      3165
           2       0.97      0.98      0.97      2784
           3       0.98      0.98      0.98      2966
           4       0.98      0.98      0.98      2805
           5       0.98      0.98      0.98      2541
           6       0.98      0.98      0.98      2788
           7       0.98      0.98      0.98      2979
           8       0.97      0.97      0.97      2732
           9       0.96      0.97      0.97      2839

[I 2023-08-14 01:25:15,384] Trial 1 finished with value: 0.9834828976459877 and parameters: {'lr': 0.0017810330702433556, 'weight_decay': 6.271757591985645e-06, 'batch_size': 42, 'epochs': 6, 'beta1': 0.8166998638814977, 'beta2': 0.929306647983458, 'factor': 0.2297691508626524, 'scheduler_patience': 1, 'threshold': 4.425069712027412e-05, 'cooldown': 1, 'min_lr': 0, 'dropout': 0.10957918637430152, 'activation': 'leaky_relu', 'test_size': 0.1830523619624917, 'random_state': 273}. Best is trial 0 with value: 0.9847794117647058.


...

|Loss Train/Valid:   0.2446  /  0.1914  |
|Accuracy Train/Valid:   92.7247%  /  94.3868%  |
-----------------------------------------------
Training Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      2400
           1       0.98      0.97      0.97      2782
           2       0.93      0.92      0.93      2465
           3       0.91      0.90      0.91      2568
           4       0.94      0.93      0.94      2425
           5       0.91      0.90      0.90      2231
           6       0.95      0.96      0.95      2417
           7       0.95      0.94      0.94      2614
           8       0.88      0.88      0.88      2355
           9       0.88      0.89      0.88      2498

    accuracy                           0.93     24755
   macro avg       0.93      0.93      0.93     24755
weighted avg       0.93      0.93      0.93     24755

Validation Classification Report:
              precision    recall  

[I 2023-08-14 01:26:32,957] Trial 3 finished with value: 0.9880840572904438 and parameters: {'lr': 3.204201922485777e-05, 'weight_decay': 0.006116774282464299, 'batch_size': 47, 'epochs': 6, 'beta1': 0.8895549087061183, 'beta2': 0.9369990853374751, 'factor': 0.11762974515228268, 'scheduler_patience': 2, 'threshold': 3.196173824327435e-05, 'cooldown': 0, 'min_lr': 0, 'dropout': 0.22840716750332216, 'activation': 'leaky_relu', 'test_size': 0.40560353073372113, 'random_state': 338}. Best is trial 3 with value: 0.9880840572904438.


|Loss Train/Valid:   0.0029  /  0.0463  |
|Accuracy Train/Valid:   99.9239%  /  98.8084%  |
-----------------------------------------------

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
MODEL HAVE BEEN SAVED BEAWARE
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2478
           1       1.00      1.00      1.00      2772
           2       1.00      1.00      1.00      2475
           3       1.00      1.00      1.00      2598
           4       1.00      1.00      1.00      2411
           5       1.00      1.00      1.00      2250
           6       1.00      1.00      1.00      2452
           7       1.00      1.00      1.00      2625
           8       1.00      1.00      1.00      2453
           9       1.00      1.00      1.00      2450

[I 2023-08-14 01:27:28,793] Trial 4 finished with value: 0.09817062126426372 and parameters: {'lr': 0.00839890469525455, 'weight_decay': 2.574719927822453e-05, 'batch_size': 45, 'epochs': 10, 'beta1': 0.8451051998105569, 'beta2': 0.9068682328203357, 'factor': 0.12517644716009996, 'scheduler_patience': 2, 'threshold': 7.536735450984983e-05, 'cooldown': 0, 'min_lr': 0, 'dropout': 0.2555453316397311, 'activation': 'sigmoid', 'test_size': 0.394345133470221, 'random_state': 1010}. Best is trial 3 with value: 0.9880840572904438.


|Loss Train/Valid:   2.7214  /  2.6575  |
|Accuracy Train/Valid:   9.9855%  /  9.8171%  |
-----------------------------------------------
Training Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.10      0.10      2535
           1       0.12      0.13      0.12      2874
           2       0.11      0.11      0.11      2617
           3       0.10      0.10      0.10      2605
           4       0.09      0.09      0.09      2446
           5       0.09      0.08      0.08      2297
           6       0.09      0.09      0.09      2432
           7       0.10      0.11      0.11      2655
           8       0.10      0.09      0.09      2433
           9       0.10      0.10      0.10      2543

    accuracy                           0.10     25437
   macro avg       0.10      0.10      0.10     25437
weighted avg       0.10      0.10      0.10     25437

Validation Classification Report:
              precision    recall  f1

In [ ]:

test_dataloader=create_dataloader(dfTest, batch_size=best_params['batch_size'], is_train=False)
tuned_model=tuned_model.to(device)
test_predictions = predict(tuned_model, test_dataloader,device)
create_and_download_submission(test_predictions, 'submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_predictions = predict(tuned_model, test_dataloader,device)
create_and_download_submission(test_predictions, 'submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_predictions[0]

2

In [ ]:
test_predictions

[2,
 0,
 9,
 9,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 4,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 4,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 9,
 9,
 2,
 3,
 5,
 4,
 1,
 0,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 4,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 8,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 3,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 7,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 3,
 6,
 4,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 3,
 7,
 6,
 3,
 0,
 2,
 7,
 8,
 6,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 8,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 3,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 0,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 5,


In [ ]:
from sklearn.metrics import classification_report

def train(model, optimizer, train_dataloader, device, loss_function):
    model.train()
    total_loss = 0
    correct_predictions = 0
    true_labels_train = []  # Store true labels
    predicted_labels_train = []  # Store predicted labels

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        model.zero_grad()
        predictions = model(images)
        loss = loss_function(predictions, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted_labels = torch.max(predictions, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        true_labels_train.extend(labels.cpu().numpy())  # Add true labels to list
        predicted_labels_train.extend(predicted_labels.cpu().numpy())  # Add predicted labels to list

    avg_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / len(train_dataloader.dataset)

    return avg_loss, accuracy, true_labels_train, predicted_labels_train

def evaluate(model, val_dataloader, device, loss_function):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    true_labels_eval = []  # Store true labels
    predicted_labels_eval = []  # Store predicted labels

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            predictions = model(images)
            loss = loss_function(predictions, labels)
            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_labels = torch.max(predictions, 1)
            correct_predictions += (predicted_labels == labels).sum().item()
            true_labels_eval.extend(labels.cpu().numpy())  # Add true labels to list
            predicted_labels_eval.extend(predicted_labels.cpu().numpy())  # Add predicted labels to list

    avg_loss = total_loss / len(val_dataloader)
    accuracy = correct_predictions / len(val_dataloader.dataset)

    return avg_loss, accuracy, true_labels_eval, predicted_labels_eval

best_state_dict = None
best_params = None
best_valid_metric = 0
def objective(trial, dfTrain, device, layers=3):
    global best_state_dict, best_params,best_valid_metric

    model_params = {
    "lr": [3e-5, 1e-2],
    "weight_decay": [1e-6, 1e-2],
    "batch_size": [40, 50],
    "epochs": [5,10],
    "beta1": [0.8, 0.99],
    "beta2": [0.9, 0.9999],
    "factor": [0.05, 0.5],
    "scheduler_patience": [1, 2],
    "threshold": [1e-5, 1e-3],
    "cooldown": [0, 1],
    "min_lr": [0, 1e-3],
    "dropout": [0.1, 0.5],  # Only one definition of "dropout" is needed
    "activation": ['leaky_relu', 'sigmoid', 'tanh', 'gelu'],
    'test_size':[0.15,0.45],
    'random_state':[42,1024],
    }
    params = {}
    for param, bounds in model_params.items():
        if isinstance(bounds[0], int):
            params[param] = trial.suggest_int(param, min(bounds), max(bounds))
        elif isinstance(bounds[0], bool) or isinstance(bounds[0], str): # Handle strings
            params[param] = trial.suggest_categorical(param, bounds)
        elif isinstance(bounds[0], float):
            params[param] = trial.suggest_float(param, min(bounds), max(bounds), log=True)

    for param, value in params.items():
        print(f'{param} = {value}')

    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    train_data, val_data = train_test_split(dfTrain, test_size=params["test_size"], random_state=params["random_state"])
    y_train,y_val=train_test_split(dfTrain['label'], test_size=params["test_size"], random_state=params["random_state"])
    train_dataloader = create_dataloader(train_data, batch_size=params["batch_size"])
    val_dataloader = create_dataloader(val_data, batch_size=params["batch_size"])  # Using the same function for validation data

    model = CNN_Arch( activation=params["activation"],dropout=params["dropout"])
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=params["lr"], betas=(params["beta1"], params["beta2"]), weight_decay=params["weight_decay"])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=params["factor"], patience=params["scheduler_patience"], verbose=False, threshold=params["threshold"], threshold_mode='rel', cooldown=params["cooldown"], min_lr=params["min_lr"])


    train_labels, val_labels = train_test_split(dfTrain['label'], test_size=params["test_size"],random_state=params["random_state"]+random.randint(0,300),stratify=dfTrain['target'])
    #all_labels = np.concatenate([train_labels, val_labels])
    all_labels=train_labels
    class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(all_labels), y=all_labels)
    weights = torch.tensor(class_weights, dtype=torch.float)
    weights = weights.to(device)




    best_local_metric = 12312310
    valid_losses = []
    no_improvement_counter = 0
    for epoch in range(1000000):
        train_loss, train_accuracy, true_labels_train, predicted_labels_train = train(model, optimizer, train_dataloader, device, loss_function)
        valid_loss, val_accuracy, true_labels_eval, predicted_labels_eval = evaluate(model, val_dataloader, device, loss_function)
        print('-' * 100); print(f'| Epoch {epoch + 1} | Loss Train/Valid: {train_loss:.6f} / {valid_loss:.6f} |', end=''); print(f' Accuracy Train/Valid: {100 * train_accuracy:.3f}% / {100 * val_accuracy:.4f}% |'); print('-' * 100);
        metric = valid_loss
        if metric < best_local_metric:
            best_local_metric = metric
            no_improvement_counter = 0
            if metric < best_valid_metric:
                best_valid_metric = metric
                best_state_dict = copy.deepcopy(model.state_dict())
                best_params=params
                print();print('~'*26);print(f'|Best Value Found {best_local_metric:.5f}|');print('~'*26); print(f'Params = {best_params}'); print('~' * 54)
                print("Training Classification Report:"); print(classification_report(true_labels_train, predicted_labels_train))
                print("Validation Classification Report:"); print(classification_report(true_labels_eval, predicted_labels_eval))
                print('~' * 54); print()
        else:
            no_improvement_counter += 1
            if no_improvement_counter >= 2:
                print(); print('<'*50); print('OH NO WE BE OVERFITTING'); print('<'*50); print()
                raise optuna.TrialPruned()
    return metric



def tune_model_with_optuna(dfTrain, device, layers=3):
    global best_state_dict, best_params

    study = optuna.create_study(direction="minimize")  # Maximize the accuracy
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=5)
    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    # Load the best model using the best state dictionary
    best_model = CNN_Arch(
        dropout=best_params["dropout"] ,
        activation=best_params["activation"],# Specify other parameters as needed
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)

    return best_model

